In [89]:
import freud, mdtraj
import MDAnalysis as mda
import matplotlib.pyplot as plt

Postprocessing

In [119]:
# Tell MDAnalysis to parse your .dat as a DATA file with “id type x y z” columns:
u = mda.Universe(
    'graphene.dat',
    topology_format='DATA',
    atom_style='id type x y z'
)

with mda.Writer('graphene_topology2.pdb', n_atoms=u.atoms.n_atoms) as W:
    W.write(u.atoms)

In [ ]:
# ──────────────────────────────────────────────────────────────────────
def minimum_image_vec(dxy, Lx, Ly):
    dxy[:, 0] -= np.round(dxy[:, 0] / Lx) * Lx
    dxy[:, 1] -= np.round(dxy[:, 1] / Ly) * Ly
    return dxy

# ──────────────────────────────────────────────────────────────────────
def compute_psi6(tree, pos, Lx, Ly, r_cut):
    """ψ6(i) with the *same* cKDTree used later for pairs."""
    nbr_lists = tree.query_ball_point(pos, r_cut)
    psi6 = np.empty(len(pos), dtype=np.complex128)

    for i, nbrs in enumerate(nbr_lists):
        if i in nbrs:
            nbrs.remove(i)
        if not nbrs:
            psi6[i] = 0.0
            continue
        vecs = pos[nbrs] - pos[i]
        vecs = minimum_image_vec(vecs, Lx, Ly)
        theta = np.arctan2(vecs[:, 1], vecs[:, 0])
        psi6[i] = np.exp(1j * 6.0 * theta).mean()
    return psi6

# ──────────────────────────────────────────────────────────────────────
def g6(pos, dr=0.1, r_max=None, r_cut=1.2, min_counts=75):
    # ----- basic box --------------------------------------------------
    Lx, Ly = pos[:, 0].ptp()+1, pos[:, 1].ptp()+1
    if r_max is None:
        r_max = 0.5 * min(Lx, Ly)

    # ----- single cKDTree for everything ------------------------------
    tree = cKDTree(pos)

    psi6       = compute_psi6(tree, pos, Lx, Ly, r_cut)
    psi6_mag2  = (np.abs(psi6) ** 2).mean()

    # ----- vectorised pair list up to r_max ---------------------------
    pairs = np.array(list(tree.query_pairs(r_max)))
    dxy   = pos[pairs[:, 1]] - pos[pairs[:, 0]]
    dxy   = minimum_image_vec(dxy, Lx, Ly)
    r     = np.hypot(dxy[:, 0], dxy[:, 1])

    # ----- histogram in one pass -------------------------------------
    nbins   = int(np.ceil(r_max / dr))
    bin_idx = np.floor(r / dr).astype(np.int64)

    num    = np.zeros(nbins, dtype=np.complex128)
    counts = np.bincount(bin_idx, minlength=nbins)

    weights = psi6[pairs[:, 0]] * np.conj(psi6[pairs[:, 1]])
    np.add.at(num, bin_idx, weights)            # works with complex

    # ----- build g6(r) ------------------------------------------------
    r_cent = (np.arange(nbins) + 0.5) * dr
    mask   = counts >= min_counts
    g6_r   = np.zeros(nbins)
    g6_r[mask] = np.abs(num[mask] / counts[mask]) / psi6_mag2
    return r_cent[mask], g6_r[mask]

In [ ]:
a = 1.42
plt.figure()

for lbl, fn, c, txt, cut in [('crystal','data/5k_atoms2.txt',"b",False,1.2),
                             ('hexatic','data/5k_atoms.txt',"lime",False,1.2)]:
    pos = np.loadtxt(fn) if txt else np.loadtxt(fn)[:,2:4]
    r, g = g6(pos/a, dr=0.05, r_cut=cut, min_counts=1)
    g_sm = gaussian_filter1d(g, sigma=2)
    plt.loglog(r[:-3]/a, g_sm[:-3], label=lbl, c=c)

    if lbl == 'liquid':
        r_env, g_env, (A, xi) = envelope_fit(r/a, g_sm)
        r_line = np.linspace(r_env.min(), r_env.max(), 200)
        plt.loglog(r_line, A*np.exp(-r_line/xi), '--', c="r",
                   label = r'$e^{-r/ \xi_6}, \,\xi = $' + f'={xi:.2f} a')
        
plt.loglog(r/a, 0.9*r**(-1/4), c='lime', label=r"$\sim r^{-1/4}$")
plt.xlabel(r'$r/a$')
plt.ylabel(r'$g_6(r)$')
plt.xlim(1, 60/a)
plt.ylim(1e-2, 1)
plt.legend()
plt.tight_layout()
plt.show()

In [5]:
# polycrystalline_graphene.py
"""
Polycrystalline graphene generator **v3** — now with a *robust* g₆(r)
calculation that:

1.  Treats the simulation box as genuinely periodic by using the known
    domain length **L** (no oversized buffer hacks).
2.  Normalises with `|⟨ψ₆(i)ψ₆*(j)⟩|` so the correlation is strictly
    non‑negative (eliminates the oscillatory sign flips you called
    “nonsense”).
3.  Masks bins with too‑few pair counts (configurable `--minpairs`) to
    avoid the late‑r noise hump.

Usage examples
--------------
```bash
# quick look, default parameters
python polycrystalline_graphene.py

# finer g₆ bins, stricter masking, save figures
python polycrystalline_graphene.py --dr 0.15 --minpairs 200 \
                                   --out struct.png g6.png
```

Dependencies: numpy, scipy, matplotlib, shapely (optional for clipping).
"""
from __future__ import annotations

import argparse
import math
import sys
from dataclasses import dataclass
from typing import Tuple

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, cKDTree

try:
    from shapely.geometry import Polygon, Point
    from shapely.ops import clip_by_rect
    USE_SHAPELY = True
except ImportError:
    USE_SHAPELY = False

###############################################################################
# Voronoi helper ──────────────────────────────────────────────────────────────
###############################################################################

def voronoi_finite_polygons_2d(vor: Voronoi, radius: float | None = None):
    """Return finite Voronoi regions (list of vertex‑index lists) & vertices."""
    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2‑D input")
    new_regions: list[list[int]] = []
    new_vertices = vor.vertices.tolist()
    center = vor.points.mean(axis=0)
    radius = radius or vor.points.ptp().max() * 2
    ridge_map: dict[int, list[Tuple[int, int, int]]] = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        ridge_map.setdefault(p1, []).append((p2, v1, v2))
        ridge_map.setdefault(p2, []).append((p1, v1, v2))
    for p1, region_idx in enumerate(vor.point_region):
        verts = vor.regions[region_idx]
        if -1 not in verts:
            new_regions.append(verts)
            continue
        region = [v for v in verts if v != -1]
        for p2, v1, v2 in ridge_map[p1]:
            if v2 == -1:
                v1, v2 = v2, v1
            if v1 != -1:
                continue
            t = vor.points[p2] - vor.points[p1]
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])
            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius
            new_vertices.append(far_point.tolist())
            region.append(len(new_vertices) - 1)
        new_regions.append(region)
    return new_regions, np.asarray(new_vertices)

###############################################################################
# Lattice generation ──────────────────────────────────────────────────────────
###############################################################################

def triangular_lattice(a: float, bbox: Tuple[float, float, float, float],
                       angle: float, origin: np.ndarray) -> np.ndarray:
    """Triangular lattice points inside *bbox* (axis‑aligned clip)."""
    e1 = np.array([a, 0.0])
    e2 = np.array([a / 2, a * math.sqrt(3) / 2])
    R = np.array([[math.cos(angle), -math.sin(angle)],
                  [math.sin(angle),  math.cos(angle)]])
    e1, e2 = R @ e1, R @ e2

    M = np.column_stack((e1, e2))
    Minv = np.linalg.inv(M)
    xmin, xmax, ymin, ymax = bbox
    corners = np.array([[xmin, ymin], [xmax, ymin], [xmin, ymax], [xmax, ymax]])
    nm = (Minv @ (corners.T - origin.reshape(2, 1))).T
    n_min, m_min = np.floor(nm.min(axis=0)) - 1
    n_max, m_max = np.ceil(nm.max(axis=0)) + 1
    n_vals = np.arange(int(n_min), int(n_max) + 1)
    m_vals = np.arange(int(m_min), int(m_max) + 1)
    n_grid, m_grid = np.meshgrid(n_vals, m_vals, indexing="ij")
    pts = origin + n_grid[..., None] * e1 + m_grid[..., None] * e2
    pts = pts.reshape(-1, 2)
    mask = ((xmin <= pts[:, 0]) & (pts[:, 0] <= xmax) &
            (ymin <= pts[:, 1]) & (pts[:, 1] <= ymax))
    return pts[mask]

###############################################################################
# g6 with proper PBC & bin masking ───────────────────────────────────────────
###############################################################################

def _minimum_image_vec(dxy: np.ndarray, L: float) -> np.ndarray:
    dxy -= np.round(dxy / L) * L
    return dxy


def _compute_psi6(pos: np.ndarray, L: float, tree: cKDTree, r_cut: float) -> np.ndarray:
    nbr_list = tree.query_ball_point(pos, r_cut)
    psi6 = np.empty(len(pos), dtype=np.complex128)
    for i, nbrs in enumerate(nbr_list):
        if i in nbrs:
            nbrs.remove(i)
        if not nbrs:
            psi6[i] = 0.0
            continue
        vecs = pos[nbrs] - pos[i]
        vecs = _minimum_image_vec(vecs, L)
        theta = np.arctan2(vecs[:, 1], vecs[:, 0])
        psi6[i] = np.exp(1j * 6 * theta).mean()
    return psi6


def g6(pos: np.ndarray, L: float, dr: float = 0.2, r_cut: float = 1.2,
       minpairs: int = 100):
    """Return r, g₆(r) with |…| and bin masking."""
    tree = cKDTree(pos)
    psi6 = _compute_psi6(pos, L, tree, r_cut)
    psi6_norm = (np.abs(psi6) ** 2).mean()

    r_max = 0.55 * L
    pairs = np.array(list(tree.query_pairs(r_max)))
    dxy = pos[pairs[:, 1]] - pos[pairs[:, 0]]
    dxy = _minimum_image_vec(dxy, L)
    r = np.hypot(dxy[:, 0], dxy[:, 1])

    nbins = int(np.ceil(r_max / dr))
    bin_idx = np.floor(r / dr).astype(int)
    num = np.zeros(nbins, dtype=np.complex128)
    counts = np.bincount(bin_idx, minlength=nbins)
    np.add.at(num, bin_idx, psi6[pairs[:, 0]] * np.conj(psi6[pairs[:, 1]]))

    r_cent = (np.arange(nbins) + 0.5) * dr
    g6_r = np.full(nbins, np.nan)
    valid = counts >= minpairs
    g6_r[valid] = np.abs(num[valid] / counts[valid]) / psi6_norm
    return r_cent, g6_r, counts

###############################################################################
# Main pipeline ──────────────────────────────────────────────────────────────
###############################################################################

@dataclass
class PolyCrystal:
    pos: np.ndarray
    r: np.ndarray
    g6: np.ndarray
    counts: np.ndarray


def build_polycrystal(n_seeds: int = 25, L: float = 30.0, a: float = 1.0,
                      dr: float = 0.2, r_cut: float = 1.2, minpairs: int = 100,
                      seed: int | None = None) -> PolyCrystal:
    rng = np.random.default_rng(seed)
    seeds = rng.random((n_seeds, 2)) * L
    angles = rng.random(n_seeds) * 2 * math.pi
    vor = Voronoi(seeds)
    regions, verts = voronoi_finite_polygons_2d(vor)

    from matplotlib.path import Path as MplPath
    atoms: list[np.ndarray] = []
    for s, ang, reg in zip(seeds, angles, regions):
        poly = verts[reg]
        if USE_SHAPELY:
            poly = np.asarray(clip_by_rect(Polygon(poly), 0, 0, L, L).exterior.coords)
        else:
            poly = np.clip(poly, 0, L)
        xmin, xmax = poly[:, 0].min(), poly[:, 0].max()
        ymin, ymax = poly[:, 1].min(), poly[:, 1].max()
        pts = triangular_lattice(a, (xmin, xmax, ymin, ymax), ang, s)
        inside = (np.asarray([Polygon(poly).contains(Point(p)) for p in pts])
                  if USE_SHAPELY else MplPath(poly).contains_points(pts))
        atoms.append(pts[inside])
    pos = np.vstack(atoms) % L  # ensure 0≤x<
